## Méthode des Trapèzes

![trapeze]("figures/trapeze_rule.png")

On sait que l'aire d'un trapèze est donnée par : $$Aire = \frac{(B + b)h}{2}$$

On définit : $$W_i = \frac{(f(x_i) + f(x_{i+1}))h}{2}$$

On trouve une formule pour approximer notre intégrale :

$$
\begin{align*}
    I &\approx \sum_{i=0}^{n-1} W_i \\
      &\approx \sum_{i=0}^{n-1} \frac{h}{2}(f(x_i) + f(x_{i+1}) \\ 
      &\approx \frac{h}{2}(f(x_0) + f(x_1) + f(x_1) + ... + f(x_{n-1}) + f(x_{n-1}) + f(x_n)) \\
      &\approx \frac{h}{2} \left[ f(x_0) + 2 \sum_{i=0}^{n-1}f(x_i) + f(x_n)) \right] \\
      &\approx h \left[ \frac{1}{2}(f(x_0) + f(x_n)) + \sum_{i=0}^{n-1}f(x_i)) \right]
\end{align*}
$$

### Itération 
- On diminue progressivement le pas de h de $\frac{1}{2}$ à chaque itération jusqu'à atteindre la précision souhaitée.
- /!\ une itération = une évaluation complète de l'intégrale.

Pour une fonction : $$\int_a^b f(x)dx$$

$n = 2^{k-1}$ => 1, 2, 4, 8,...

$h = \frac{b-a}{n}$ => $h$, $\frac{h}{2}$, $\frac{h}{4}$,...

Inconvénient : convergence lente donc besoin de beaucoup d'itérations pour être précis.

In [1]:
import numpy as np
import math

# precision de l'évaluation numérique de l'intégrale (trapèzes)
TRESHOLD_PRECISION = 1e-9
DIMENSIONS = [1] #[1, 2, 10, 15]
X_0 = -3 
X_N = 3

class Trapezes():
  def __init__(self, f, x_0, x_n):
    self.f = f
    self.x_0 = x_0
    self.x_n = x_n
  
  # borne inférieure : x_0
  # borne supérieure : x_n
  # en pratique on itère sur la méthode 
  # et on diminue le pas h progressivement (de 1/2 à chaque itération)
  # k : itération en cours
  def trapezes(self, k):
    # si k = 1, le nombre de trapèzes serait de 1 (en effet, 2^0 = 1)
    if k == 1: 
      # Aire d'un trapèze
      h = (self.x_n - self.x_0)
      return (self.f(self.x_0) + self.f(self.x_n)) * h / 2

    # nombre de trapèzes
    n = int(math.pow(2, k-1))
    # intervalle h
    h = (self.x_n - self.x_0) / n

    sum = 1/2 * (self.f(self.x_0) + self.f(self.x_n))
    for i in range(1, n):
      x_i = self.x_0 + (i * h)
      sum += self.f(x_i)

    return h * sum 
  
  # N : nombre de dimensions
  def integrate(self, iterations, precision):
    new_integral = 0
    for k in range(1, iterations):
      old_integral = new_integral
      new_integral = self.trapezes(k)
      if abs(old_integral - new_integral) < precision and k > 1:
        return new_integral, k
        break

def gaussian(x):
  f = math.exp(-(x**2) / 2)
  return f

def gaussian_2_dim(x, y):
  return math.exp(-(x**2 + y**2) / 2)

def gaussian_theo(N):
  return math.sqrt((2*math.pi)**N)

print(f"dimension 1")
print("-----------")
print(f"Valeur théorique approximative: {gaussian_theo(N=1)}")

gaussian_trapezes = Trapezes(gaussian, X_0, X_N)
gaussian_trapezes_value = gaussian_trapezes.integrate(iterations = 20, precision = TRESHOLD_PRECISION)
print(f"Valeur de la gaussienne à 1 dimension (trapèzes): {gaussian_trapezes_value}")

print(f"dimension 2")
print("-------------")
print(f"Valeur théorique approximative: {gaussian_theo(N=2)}")



dimension 1
-----------
Valeur théorique approximative: 2.5066282746310002
Valeur de la gaussienne à 1 dimension (trapèzes): (2.4998608892968623, 16)
dimension 2
-------------
Valeur théorique approximative: 6.283185307179586
